# Code Generation and Evaluation

__TABLE 1. AVAILABLE MISTRAL AI MODELS__
| Name                        | # Parameters | # Active Parameters | Min. GPU RAM for inference (GB) | API Endpoint          | Comments   |   Official HumanEval  |  My Result  |
| --------------------------- | ------------ | ------------------- | ------------------------------- | --------------------- | ------------------------------- |  ---------------  |  ---------------------  |
| __MODELS I PLAN TO USE:__ |
| Mistral-7B-v0.3             | 7.3B         | 7.3B                | 16                              | open-mistral-7b       |                     |  30.5%  | 31.1%      |
| Mixtral-8x7B-v0.1           | 46.7B        | 12.9B               | 100                             | open-mixtral-8x7b     |       |  40.2%  | 16.46%  |                                                       |
| Ministral-3B-2410           | 3B           | 3B                  |                                 | ministral-3b-latest   | currently points to ministral-3b-2410.<br>World’s best edge model. |  77.4% (instruct) | 63.64%
| Ministral-8B-2410           | 8B           | 8B                  | 24                              | ministral-8b-latest   | currently points to ministral-8b-2410                        |  76.8%  |  72.56%  ||
| Codestral-Mamba-7B-v0.1     | 7.3B         | 7.3B                | 16                              | open-codestral-mamba  |       |     75% |  75.61% |
| Mistral-Nemo-Instruct-2407  | 12B          | 12B                 | 28 - bf16,<br>16 - fp8          | open-mistral-nemo     | currently points to open-mistral-nemo-2407  |  67%  | 58.54% |
| Codestral-22B-v0.1          | 22.2B        | 22.2B               | 60                              |  codestral-latest                     |  currently points to codestral-2405   |  81.1% |  26.83% | 
| Mistral-Small-2409          | 22B          | 22B                 | 60                              | mistral-small-latest                      |  currently points to mistral-small-2409  |  80%  |  70.73%                                                          |
|           |           |                  |                               |                       |                                                              |
| __MODELS I DON'T PLAN TO USE (NOT AN SLM OR IRRELEVANT):__ |
| Mathstral-7B-v0.1           | 7.3B         | 7.3B                | 16                              |                       |                                                              |
| Mixtral-8x22B-v0.3          | 140.6B       | 39.1B               | 300                             |                       |                                                              |
| Pixtral-2409                | 12B          | 12B                 | 28 - bf16,<br>16 - fp8          |                       |                                                              |
| Mistral-Large-Instruct-2407 | 123B         | 123B                | 250                             |                       |                                                              |
| Mistral-Large-Instruct-2411 | 123B         | 123B                | 250                             |                       |                                                              |
| Pixtral-Large-Instruct-2411 | 124B         | 124B                | 250                             |                       |                                                              |

__Notes__:  
* Use the API endpoint name to make API calls.
* Rate limits: 2M tokens/minute, 10B tokens month

__Sources__:
1) https://docs.mistral.ai/getting-started/models/weights/
2) https://docs.mistral.ai/getting-started/models/models_overview/
3) For official HumanEval and MBPP scores, click on each model's blog post link (in link 2)

In [1]:
from human_eval.data import write_jsonl, read_problems
from human_eval.evaluation import evaluate_functional_correctness
import os
import logging
import sys
import time, datetime
from mistralai import Mistral
import json

%load_ext autoreload
%autoreload 2

In [2]:
def remove_triple_backticks_at_edges(text: str) -> str:
    """
    Removes triple backticks (` ``` `) if they appear at the beginning or the end of the text.
    :param text: The input text as a string.
    :return: A string with triple backticks removed from the edges.
    """
    text = text.strip()
    s1 = "```python\n"
    s2 = "```python"
    s3 = "```"
    for s in [s1, s2, s3]:
        if text.startswith(s):
            text = text[len(s):].lstrip()      # Remove a version of backticks + extra whitespace at the start
    if text.endswith(s3):
        text = text[: -len(s3)].rstrip()       # Remove the backticks + extra whitespace at the end
    return text


def add_import(text: str) -> str:
    ''' Adds import statements stripped by LLM '''
    text = text.strip()
    s = 'from typing import List'
    if s not in text:
        text = s + '\n\n' + text
    return text

string = 'def my_func(input):\n    return output'
print(add_import(string))

from typing import List

def my_func(input):
    return output


In [3]:
def get_response(model, prompt):
    ''' One API call to a Mistral model '''
    chat_response = client.chat.complete(
                model = model,
                messages = [{
                        "role": "user",
                        "content": prompt, 
                    }, ]
                )
    return chat_response.choices[0].message.content

In [4]:
def read_jsonl(file_path):
    '''
    Read a JSONL file.
    Parameters:
        file_path (str): The path to the JSONL file.
    Returns:
        List[dict]: A list of dictionaries representing the JSON objects.
    '''
    data = []
    with open(file_path) as f:
        for line in f:
            line = line.strip()
            if line:
                data.append(json.loads(line))
    return data

In [5]:
tasks = read_problems()
print(f'Type of tasks: {type(tasks)}\n')

for k,v in tasks.items():
    print(f'task_id: {k}')
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')

Type of tasks: <class 'dict'>

task_id: HumanEval/0

task_id:
HumanEval/0

prompt:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """


entry_point:
has_close_elements

canonical_solution:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False


test:


METADATA = {
    'author': 'jt',
    'dataset': 'test'
}


def check(candidate):
    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
    assert candidate([1.0, 2.0, 5.9, 

In [6]:
# modified prompt for codestral (worse results fs. regular prompt)
prompt_prefix2 = '''1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Exclude clarifications: do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Exclude any code or text after the function. Do not repeat the function twice. Exclude usage examples. Exclude test cases. Do not try to test or run the function.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:\n
{}
'''

# regular prompt used for most of Mistral models
prompt_prefix = '''1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Do not include any explanations or clarifications. Do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Do not add anything after the function. Do not repeat the function twice. Do not provide usage examples.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:\n
{}
'''
sample_prompt = tasks['HumanEval/0']['prompt']
print(prompt_prefix.format(sample_prompt))

1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Do not include any explanations or clarifications. Do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Do not add anything after the function. Do not repeat the function twice. Do not provide usage examples.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """




In [7]:
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

In [14]:
mistral_7b = 'open-mistral-7b'
mistral_3b = 'ministral-3b-latest'
mistral_8B = 'ministral-8b-latest'
codestral_mamba = 'open-codestral-mamba'
codestral = 'codestral-latest'
nemo = 'open-mistral-nemo'
mistral_small = 'mistral-small-latest'
mixtral = 'open-mixtral-8x7b'

model = mixtral
print(get_response(model, prompt_prefix.format(sample_prompt)))

from typing import List

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for i in range(len(numbers) - 1):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False

------------------------------------------------------------------

from typing import List

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for i in range(len(numbers) - 1):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False


In [18]:
model

'open-mixtral-8x7b'

In [19]:
num_samples_per_task = 1
completions = []
print(f'Model: {model}\n')

for task_id, task_body in tasks.items():
    print(task_id)
    #if task_id == 'HumanEval/15':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        full_prompt = prompt_prefix.format(task_body["prompt"])
        #full_prompt = task_body["prompt"]
        try:
            completion = get_response(model, full_prompt)
        except Exception as e:
            completion = f"Error when model predicted a completion:\n{e}"
        
        completion = remove_triple_backticks_at_edges(completion)
        completion = add_import(completion)
        completions.append( {'task_id': task_id, 'completion': completion} )
        print(full_prompt, '\n', '='*75, '\n', sep='')
        print(completion)
        print(f"\nTime elapsed: {(time.time() - start_time):.4f} seconds\n", '\n', '='*75, '\n', sep='')

Model: open-mixtral-8x7b

HumanEval/0
1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Do not include any explanations or clarifications. Do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Do not add anything after the function. Do not repeat the function twice. Do not provide usage examples.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    Tr

1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Do not include any explanations or clarifications. Do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Do not add anything after the function. Do not repeat the function twice. Do not provide usage examples.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:

from typing import List


def intersperse(numbers: List[int], delimeter: int) -> List[int]:
    """ Insert a number 'delimeter' between every two consecutive elements of input list `numbers'
    >>> intersperse([], 4)
    []
    >>> intersperse([1, 2, 3], 4)
    [1, 4, 2, 4, 3]
    """



from typing import List

def intersperse(numbers: List[int], delimeter: int) -> List[

1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Do not include any explanations or clarifications. Do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Do not add anything after the function. Do not repeat the function twice. Do not provide usage examples.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:



def is_palindrome(string: str) -> bool:
    """ Test if given string is a palindrome """
    return string == string[::-1]


def make_palindrome(string: str) -> str:
    """ Find the shortest palindrome that begins with a supplied string.
    Algorithm idea is simple:
    - Find the longest postfix of supplied string that is a palindrome.
    - Append to the end of the s

1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Do not include any explanations or clarifications. Do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Do not add anything after the function. Do not repeat the function twice. Do not provide usage examples.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:



def string_sequence(n: int) -> str:
    """ Return a string containing space-delimited numbers starting from 0 upto n inclusive.
    >>> string_sequence(0)
    '0'
    >>> string_sequence(5)
    '0 1 2 3 4 5'
    """



from typing import List

import string

def string\_sequence(n: int) -> str:
`pad` = ' '.join(map(str,range(n+1)))
    return pad

string\_sequence = str

1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Do not include any explanations or clarifications. Do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Do not add anything after the function. Do not repeat the function twice. Do not provide usage examples.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:

from typing import List


def rescale_to_unit(numbers: List[float]) -> List[float]:
    """ Given list of numbers (of at least two elements), apply a linear transform to that list,
    such that the smallest number will become 0 and the largest will become 1
    >>> rescale_to_unit([1.0, 2.0, 3.0, 4.0, 5.0])
    [0.0, 0.25, 0.5, 0.75, 1.0]
    """



from typing import Lis

...

1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Do not include any explanations or clarifications. Do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Do not add anything after the function. Do not repeat the function twice. Do not provide usage examples.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:


def fix_spaces(text):
    """
    Given a string text, replace all spaces in it with underscores, 
    and if a string has more than 2 consecutive spaces, 
    then replace all consecutive spaces with - 
    
    fix_spaces("Example") == "Example"
    fix_spaces("Example 1") == "Example_1"
    fix_spaces(" Example 2") == "_Example_2"
    fix_spaces(" Example   3") == "_Ex

1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Do not include any explanations or clarifications. Do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Do not add anything after the function. Do not repeat the function twice. Do not provide usage examples.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:


def order_by_points(nums):
    """
    Write a function which sorts the given list of integers
    in ascending order according to the sum of their digits.
    Note: if there are several items with similar sum of their digits,
    order them based on their index in original list.

    For example:
    >>> order_by_points([1, 11, -1, -11, -12]) == [-1, -11, 1, -12, 11]
   

1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Do not include any explanations or clarifications. Do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Do not add anything after the function. Do not repeat the function twice. Do not provide usage examples.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:


def x_or_y(n, x, y):
    """A simple program which should return the value of x if n is 
    a prime number and should return the value of y otherwise.

    Examples:
    for x_or_y(7, 34, 12) == 34
    for x_or_y(15, 8, 5) == 5
    
    """



from typing import List

from math import sqrt

def is\_prime(n):
if n < 2:
return False
for i in range(2, int(sqrt(n)) + 1):
if 

1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Do not include any explanations or clarifications. Do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Do not add anything after the function. Do not repeat the function twice. Do not provide usage examples.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:


def even_odd_count(num):
    """Given an integer. return a tuple that has the number of even and odd digits respectively.

     Example:
        even_odd_count(-12) ==> (1, 1)
        even_odd_count(123) ==> (1, 2)
    """



from typing import List

import re

def even_odd_count(num):
    """Given an integer. return a tuple that has the number of even and odd digits resp

1. Complete the following Python starter code.
2. Output the completed function as complete Python code, including all the required import statements, the starter code, and its completion.
3. Do not include any explanations or clarifications. Do not say "completion:", or "Here is a completion", or "Concatenated code:", or anything like that.
4. Do not add anything after the function. Do not repeat the function twice. Do not provide usage examples.
5. Ensure that the function is fully implemented and syntactically correct so that it can be executed without errors when called from another Python script.

Starter code:


def do_algebra(operator, operand):
    """
    Given two lists operator, and operand. The first list has basic algebra operations, and 
    the second list is a list of integers. Use the two given lists to build the algebric 
    expression and return the evaluation of this expression.

    The basic algebra operations:
    Addition ( + ) 
    Subtraction ( - ) 
    Multi

In [20]:
model_nickname = 'mixtral'
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
file_name      = f'logs/{model_nickname}_samples_{time_stamp}.jsonl'
write_jsonl(file_name, completions)

In [12]:
# if the above cell has not been run
#file_name = 'logs2/solar_10B_samples_20241127_232205_5243.jsonl'

In [21]:
#evaluate_functional_correctness(file_name, k=[1], use_prompt=True)
evaluate_functional_correctness(file_name, k=[1])

Reading samples...


164it [00:00, 21139.74it/s]


Running test suites...


100%|██████████████████████████████████████████████████████| 164/164 [00:09<00:00, 17.18it/s]


Writing results to logs/mixtral_samples_20241203_012620_1640.jsonl_results.jsonl...


100%|███████████████████████████████████████████████████| 164/164 [00:00<00:00, 59206.91it/s]


{'pass@1': 0.16463414634146342}

# Appendix

In [14]:
# read a jsonl file
file_name = 'logs/mistral_7b_samples_20241201_005244_0462.jsonl'
data = read_jsonl(file_name)

In [17]:
data[0]['completion']

'from typing import List\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) <= threshold:\n                return True\n    return False'

In [24]:
# modify and write back
data2 = []
for i in data:
    new_i = dict()
    new_i['task_id'] = i['task_id']
    new_i['completion'] = add_import(i['completion'])
    data2.append(new_i)

In [26]:
file_name = 'logs/mistral_7b_samples_20241201_005244_0462_v2.jsonl'
write_jsonl(file_name, data2)